# House Price Pridiction Model Using ANN

In [3]:
import pandas as pd

In [12]:
df=pd.read_csv("/Users/harshverma/Downloads/houseprice.csv",usecols=["SalePrice", "MSSubClass", "MSZoning", "LotFrontage", "LotArea",
                                         "Street", "YearBuilt", "LotShape", "1stFlrSF", "2ndFlrSF"]).dropna()
df.sample(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,YearBuilt,1stFlrSF,2ndFlrSF,SalePrice
1308,20,RM,100.0,12000,Pave,Reg,1948,1122,0,147000
571,20,RL,60.0,7332,Pave,Reg,1959,864,0,120000
222,60,RL,85.0,11475,Pave,Reg,1975,811,741,179900
332,20,RL,85.0,10655,Pave,IR1,2003,1629,0,284000
1452,180,RM,35.0,3675,Pave,Reg,2005,1072,0,145000


In [8]:
df.shape

(1201, 10)

In [9]:
df.isnull().sum()

MSSubClass     0
MSZoning       0
LotFrontage    0
LotArea        0
Street         0
LotShape       0
YearBuilt      0
1stFlrSF       0
2ndFlrSF       0
SalePrice      0
dtype: int64

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1201 entries, 0 to 1459
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MSSubClass   1201 non-null   int64  
 1   MSZoning     1201 non-null   object 
 2   LotFrontage  1201 non-null   float64
 3   LotArea      1201 non-null   int64  
 4   Street       1201 non-null   object 
 5   LotShape     1201 non-null   object 
 6   YearBuilt    1201 non-null   int64  
 7   1stFlrSF     1201 non-null   int64  
 8   2ndFlrSF     1201 non-null   int64  
 9   SalePrice    1201 non-null   int64  
dtypes: float64(1), int64(6), object(3)
memory usage: 103.2+ KB


In [11]:
for i in df.columns:
    print("column name {} and unique values {}".format(i,len(df[i].unique())))

column name MSSubClass and unique values 15
column name MSZoning and unique values 5
column name LotFrontage and unique values 110
column name LotArea and unique values 869
column name Street and unique values 2
column name LotShape and unique values 4
column name YearBuilt and unique values 112
column name 1stFlrSF and unique values 678
column name 2ndFlrSF and unique values 368
column name SalePrice and unique values 597


In [13]:
import datetime
datetime.datetime.now().year

2025

In [14]:
df['Total Years']=datetime.datetime.now().year-df['YearBuilt']

In [15]:
df.drop('YearBuilt',axis=1,inplace=True)

In [16]:
df.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', '1stFlrSF', '2ndFlrSF', 'SalePrice', 'Total Years'],
      dtype='object')

In [17]:
df.head(3)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,1stFlrSF,2ndFlrSF,SalePrice,Total Years
0,60,RL,65.0,8450,Pave,Reg,856,854,208500,22
1,20,RL,80.0,9600,Pave,Reg,1262,0,181500,49
2,60,RL,68.0,11250,Pave,IR1,920,866,223500,24


In [24]:
cat_features=['MSSubClass','MSZoning','Street','LotShape']
out_features='SalePrice'

In [18]:
df['MSSubClass'].unique()

array([ 60,  20,  70,  50, 190,  45,  90, 120,  30,  80, 160,  75, 180,
        40,  85])

In [22]:
from sklearn.preprocessing import LabelEncoder
lbl_encoders={}
lbl_encoders["MSSubClass"]=LabelEncoder()
lbl_encoders["MSSubClass"].fit_transform(df["MSSubClass"])

array([5, 0, 5, ..., 6, 0, 0], shape=(1201,))

In [26]:
from sklearn.preprocessing import LabelEncoder
lbl_encoaders={}
for feature in cat_features: 
    lbl_encoaders[feature]=LabelEncoder()
    df[feature]=lbl_encoaders[feature].fit_transform(df[feature])

In [28]:
df.head(8)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,1stFlrSF,2ndFlrSF,SalePrice,Total Years
0,5,3,65.0,8450,1,3,856,854,208500,22
1,0,3,80.0,9600,1,3,1262,0,181500,49
2,5,3,68.0,11250,1,0,920,866,223500,24
3,6,3,60.0,9550,1,0,961,756,140000,110
4,5,3,84.0,14260,1,0,1145,1053,250000,25
5,4,3,85.0,14115,1,0,796,566,143000,32
6,0,3,75.0,10084,1,3,1694,0,307000,21
8,4,4,51.0,6120,1,3,1022,752,129900,94


In [31]:
#stacking 
import numpy as np
cat_features=np.stack([df['MSSubClass'],df['MSZoning'],df['Street'],df['LotShape']],1)
cat_features

array([[5, 3, 1, 3],
       [0, 3, 1, 3],
       [5, 3, 1, 0],
       ...,
       [6, 3, 1, 3],
       [0, 3, 1, 3],
       [0, 3, 1, 3]], shape=(1201, 4))

In [35]:
#create numpy to tensors
import torch
cat_features=torch.tensor(cat_features,dtype=torch.int64)
cat_features

/var/folders/0x/3pp5q0913cx2r4hsm3zv6zdr0000gn/T/ipykernel_7082/17893507.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cat_features=torch.tensor(cat_features,dtype=torch.int64)


tensor([[5, 3, 1, 3],
        [0, 3, 1, 3],
        [5, 3, 1, 0],
        ...,
        [6, 3, 1, 3],
        [0, 3, 1, 3],
        [0, 3, 1, 3]])

In [37]:
#create continuous variable
cont_features=[]
for i in df.columns:
    if i in ['MSSubClass','MSZoning','Street','LotShape','SalePrice']:
        pass
    else:
        cont_features.append(i)

In [38]:
cont_features

['LotFrontage', 'LotArea', '1stFlrSF', '2ndFlrSF', 'Total Years']

In [39]:
#Stacking continuous variable to a tensor
cont_values=np.stack([df[i].values for i in cont_features],axis=1)
cont_values=torch.tensor(cont_values,dtype=torch.float)
cont_values

tensor([[   65.,  8450.,   856.,   854.,    22.],
        [   80.,  9600.,  1262.,     0.,    49.],
        [   68., 11250.,   920.,   866.,    24.],
        ...,
        [   66.,  9042.,  1188.,  1152.,    84.],
        [   68.,  9717.,  1078.,     0.,    75.],
        [   75.,  9937.,  1256.,     0.,    60.]])

In [40]:
#dependent feature
y=torch.tensor(df['SalePrice'].values,dtype=torch.float).reshape(-1,1)
y

tensor([[208500.],
        [181500.],
        [223500.],
        ...,
        [266500.],
        [142125.],
        [147500.]])

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1201 entries, 0 to 1459
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MSSubClass   1201 non-null   int64  
 1   MSZoning     1201 non-null   int64  
 2   LotFrontage  1201 non-null   float64
 3   LotArea      1201 non-null   int64  
 4   Street       1201 non-null   int64  
 5   LotShape     1201 non-null   int64  
 6   1stFlrSF     1201 non-null   int64  
 7   2ndFlrSF     1201 non-null   int64  
 8   SalePrice    1201 non-null   int64  
 9   Total Years  1201 non-null   int64  
dtypes: float64(1), int64(9)
memory usage: 103.2 KB


In [42]:
cat_features.shape,cont_values.shape,y.shape

(torch.Size([1201, 4]), torch.Size([1201, 5]), torch.Size([1201, 1]))

In [43]:
#embeddinfg size for categorical columns
cat_dims=[len(df[col].unique()) for col in ["MSSubClass", "MSZoning", "Street", "LotShape"]]

In [44]:
embedding_dim= [(x, min(50, (x + 1) // 2)) for x in cat_dims]

In [45]:
embedding_dim

[(15, 8), (5, 3), (2, 1), (4, 2)]

In [46]:
import torch
import torch.nn as nn
import torch.nn.functional as f
embed_representation=nn.ModuleList([nn.Embedding(inp,out) for inp,out in embedding_dim])
embed_representation

ModuleList(
  (0): Embedding(15, 8)
  (1): Embedding(5, 3)
  (2): Embedding(2, 1)
  (3): Embedding(4, 2)
)

In [47]:
cat_featuresz=cat_features[:4]
cat_featuresz

tensor([[5, 3, 1, 3],
        [0, 3, 1, 3],
        [5, 3, 1, 0],
        [6, 3, 1, 0]])

In [48]:
pd.set_option('display.max_rows', 500)
embedding_val=[]
for i,e in enumerate(embed_representation):
    embedding_val.append(e(cat_features[:,i]))

In [49]:
embedding_val

[tensor([[-0.3349,  0.0534,  0.5604,  ..., -0.6045,  0.8373, -0.9637],
         [-0.5605, -2.0278,  0.6088,  ...,  0.4781,  0.5133, -0.4873],
         [-0.3349,  0.0534,  0.5604,  ..., -0.6045,  0.8373, -0.9637],
         ...,
         [-0.2758, -0.4350, -0.6845,  ..., -1.8050, -1.0195,  0.9044],
         [-0.5605, -2.0278,  0.6088,  ...,  0.4781,  0.5133, -0.4873],
         [-0.5605, -2.0278,  0.6088,  ...,  0.4781,  0.5133, -0.4873]],
        grad_fn=<EmbeddingBackward0>),
 tensor([[-1.5103,  0.1766, -1.4320],
         [-1.5103,  0.1766, -1.4320],
         [-1.5103,  0.1766, -1.4320],
         ...,
         [-1.5103,  0.1766, -1.4320],
         [-1.5103,  0.1766, -1.4320],
         [-1.5103,  0.1766, -1.4320]], grad_fn=<EmbeddingBackward0>),
 tensor([[-0.5263],
         [-0.5263],
         [-0.5263],
         ...,
         [-0.5263],
         [-0.5263],
         [-0.5263]], grad_fn=<EmbeddingBackward0>),
 tensor([[ 0.3701, -0.0947],
         [ 0.3701, -0.0947],
         [ 0.2176, -0.

In [51]:
k=torch.cat(embedding_val,1)
k

tensor([[-0.3349,  0.0534,  0.5604,  ..., -0.5263,  0.3701, -0.0947],
        [-0.5605, -2.0278,  0.6088,  ..., -0.5263,  0.3701, -0.0947],
        [-0.3349,  0.0534,  0.5604,  ..., -0.5263,  0.2176, -0.4080],
        ...,
        [-0.2758, -0.4350, -0.6845,  ..., -0.5263,  0.3701, -0.0947],
        [-0.5605, -2.0278,  0.6088,  ..., -0.5263,  0.3701, -0.0947],
        [-0.5605, -2.0278,  0.6088,  ..., -0.5263,  0.3701, -0.0947]],
       grad_fn=<CatBackward0>)

In [52]:
dropout=nn.Dropout(.4)

In [55]:
final_embed=dropout(k)
final_embed

tensor([[-0.5582,  0.0000,  0.9339,  ..., -0.8772,  0.6168, -0.0000],
        [-0.0000, -3.3796,  1.0147,  ..., -0.0000,  0.0000, -0.1578],
        [-0.0000,  0.0890,  0.9339,  ..., -0.8772,  0.3627, -0.6800],
        ...,
        [-0.0000, -0.7249, -1.1409,  ..., -0.8772,  0.6168, -0.1578],
        [-0.9342, -0.0000,  0.0000,  ..., -0.8772,  0.6168, -0.1578],
        [-0.0000, -0.0000,  1.0147,  ..., -0.8772,  0.6168, -0.1578]],
       grad_fn=<MulBackward0>)

In [56]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class FeedForwardNN(nn.Module):

    def __init__(self, embedding_dim, n_cont, out_sz, layers, p=0.5):
        super().__init__()
        self.embeds = nn.ModuleList([nn.Embedding(inp,out) for inp,out in embedding_dim])
        self.emb_drop = nn.Dropout(p)
        self.bn_cont = nn.BatchNorm1d(n_cont)
        
        layerlist = []
        n_emb = sum((out for inp,out in embedding_dim))
        n_in = n_emb + n_cont
        
        for i in layers:
            layerlist.append(nn.Linear(n_in,i)) 
            layerlist.append(nn.ReLU(inplace=True))
            layerlist.append(nn.BatchNorm1d(i))
            layerlist.append(nn.Dropout(p))
            n_in = i
        layerlist.append(nn.Linear(layers[-1],out_sz))
            
        self.layers = nn.Sequential(*layerlist)
    
    def forward(self, x_cat, x_cont):
        embeddings = []
        for i,e in enumerate(self.embeds):
            embeddings.append(e(x_cat[:,i]))
        x = torch.cat(embeddings, 1)
        x = self.emb_drop(x)
        
        x_cont = self.bn_cont(x_cont)
        x = torch.cat([x, x_cont], 1)
        x = self.layers(x)
        return x

In [57]:
torch.manual_seed(100)
model=FeedForwardNN(embedding_dim,len(cont_features),1,[100,50],p=0.4)


In [58]:
loss_function=nn.MSELoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)

In [59]:
df.shape

(1201, 10)

In [60]:
#train test split 
batch_size=1200
test_size=int(batch_size*0.15)
train_categorical=cat_features[:batch_size-test_size]
test_categorical=cat_features[batch_size-test_size:batch_size]
train_cont=cont_values[:batch_size-test_size]
test_cont=cont_values[batch_size-test_size:batch_size]
y_train=y[:batch_size-test_size]
y_test=y[batch_size-test_size:batch_size]

In [61]:
len(train_categorical),len(test_categorical),len(train_cont),len(test_cont),len(y_train),len(y_test)

(1020, 180, 1020, 180, 1020, 180)

In [66]:
epochs=5000
final_losses=[]
for i in range(epochs):
    i=i+1
    y_pred=model(train_categorical,train_cont)
    loss=torch.sqrt(loss_function(y_pred,y_train))
    final_losses.append(loss)
    if i%10==1:
        print("Epoch number: {} and the loss : {}".format(i,loss.item()))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch number: 1 and the loss : 33368.0390625
Epoch number: 11 and the loss : 34502.46484375
Epoch number: 21 and the loss : 33629.4609375
Epoch number: 31 and the loss : 34609.75
Epoch number: 41 and the loss : 34323.28515625
Epoch number: 51 and the loss : 34715.4375
Epoch number: 61 and the loss : 33560.21875
Epoch number: 71 and the loss : 31814.634765625
Epoch number: 81 and the loss : 37170.375
Epoch number: 91 and the loss : 34631.55078125
Epoch number: 101 and the loss : 33564.53515625
Epoch number: 111 and the loss : 34347.4140625
Epoch number: 121 and the loss : 34148.98046875
Epoch number: 131 and the loss : 33176.41015625
Epoch number: 141 and the loss : 34349.765625
Epoch number: 151 and the loss : 35260.93359375
Epoch number: 161 and the loss : 33236.2265625
Epoch number: 171 and the loss : 33110.015625
Epoch number: 181 and the loss : 31474.310546875
Epoch number: 191 and the loss : 34227.6953125
Epoch number: 201 and the loss : 33508.015625
Epoch number: 211 and the loss

In [70]:
y_pred=""
with torch.no_grad():
    y_pred=model(test_categorical,test_cont)
    loss=torch.sqrt(loss_function(y_pred,y_test))
print('RMSE: {}'.format(loss))

RMSE: 47185.0078125


In [72]:
data_verify=pd.DataFrame(y_test.tolist(),columns=["Test"])

In [73]:
data_predicted=pd.DataFrame(y_pred.tolist(),columns=["Prediction"])

In [74]:
data_predicted

,Prediction
0,155972.359375
1,226435.265625
2,169931.250000
3,266463.656250
4,180085.578125
5,187867.734375
6,185962.453125
7,219400.000000
8,136885.406250
9,425262.343750


# PRIDICTION

In [76]:
final_output=pd.concat([data_verify,data_predicted],axis=1)
final_output['Difference']=final_output['Test']-final_output['Prediction']
final_output.head(10)

,Test,Prediction,Difference
0,130000.0,155972.359375,-25972.359375
1,138887.0,226435.265625,-87548.265625
2,175500.0,169931.250000,5568.750000
3,195000.0,266463.656250,-71463.656250
4,142500.0,180085.578125,-37585.578125
5,265900.0,187867.734375,78032.265625
6,224900.0,185962.453125,38937.546875
7,248328.0,219400.000000,28928.000000
8,170000.0,136885.406250,33114.593750
9,465000.0,425262.343750,39737.656250


In [77]:
torch.save(model,'HousePrice.pt')

In [78]:
torch.save(model.state_dict(),'HouseWeights.pt')

In [79]:
embs_size=[(15, 8), (5, 3), (2, 1), (4, 2)]
model1=FeedForwardNN(embs_size,5,1,[100,50],p=0.4)

In [80]:
model1.load_state_dict(torch.load('HouseWeights.pt'))

<All keys matched successfully>

In [81]:
model.eval()

FeedForwardNN(
  (embeds): ModuleList(
    (0): Embedding(15, 8)
    (1): Embedding(5, 3)
    (2): Embedding(2, 1)
    (3): Embedding(4, 2)
  )
  (emb_drop): Dropout(p=0.4, inplace=False)
  (bn_cont): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): Linear(in_features=19, out_features=100, bias=True)
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=100, out_features=50, bias=True)
    (5): ReLU(inplace=True)
    (6): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.4, inplace=False)
    (8): Linear(in_features=50, out_features=1, bias=True)
  )
)